<a href="https://colab.research.google.com/github/ShinAsakawa/2019seminar_info/blob/master/notebooks/2019si_sequential_kmnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center>
<h1>[Python で 超実習ディープラーニング](https://www.seminar-info.jp/entry/seminars/view/1/4174)</h1>
<h3><strong>実践！ 強化学習・画像認識・自然言語処理・ロボティクス</strong></h3>
</center>

<center>
![](https://www.seminar-info.jp/entry/img/logo_ov.jpg)
</center>

<div align='right'>
<a href='mailto:asakawa@ieee.org'>Shin Aasakawa</a>, all rights reserved.<br>
Date: 15/Mar/2019<br>
 MIT license
</div>

In [0]:
"""
[Original](mnist_hierarchical_rnn.py)

手書き文字データセット kmnist を系列処理とみなしてリカレントニューラルネットワークによる
系列処理で認識精度を実習する
"""
import numpy as np
import keras
from keras.layers import Dense
from keras.layers import LSTM
from keras.models import Sequential

# Training parameters.
batch_size = 32
num_classes = 10
epochs = 5

In [0]:
# データの入手が未だの場合は以下の行頭の # を削除して本セルを実行してください
#!wget http://codh.rois.ac.jp/kmnist/dataset/kmnist/kmnist-train-imgs.npz
#!wget http://codh.rois.ac.jp/kmnist/dataset/kmnist/kmnist-train-labels.npz 
#!wget http://codh.rois.ac.jp/kmnist/dataset/kmnist/kmnist-test-imgs.npz
#!wget http://codh.rois.ac.jp/kmnist/dataset/kmnist/kmnist-test-labels.npz

In [0]:
# 直下行の N=10000 は実習の都合で実際のデータより少なくして学習時間を短くするためです
# 実際には全データ，すなわち N=60000 を使って評価する必要があります。
N=10000
X_train = np.load('kmnist-train-imgs.npz')['arr_0'][:N].reshape(-1,28,28)
X_test = np.load('kmnist-test-imgs.npz')['arr_0'].reshape(-1,28,28)
y_train = np.load('kmnist-train-labels.npz')['arr_0'][:N]
y_test = np.load('kmnist-test-labels.npz')['arr_0']
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((10000, 28, 28), (10000,), (10000, 28, 28), (10000,))

In [0]:
# 入出力を keras に入力できるように整形します
x_train = X_train.astype('float32')
x_test = X_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Converts class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (10000, 28, 28)
10000 train samples
10000 test samples


In [0]:
model = Sequential()   # モデルの作成
model.add(LSTM(32, input_shape=(28, 28), return_sequences=True))
model.add(LSTM(16, return_sequences=False))
model.add(Dense(num_classes, activation='softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [0]:
model.summary()  # 作成したモデルほ表示

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_5 (LSTM)                (None, 28, 32)            7808      
_________________________________________________________________
lstm_6 (LSTM)                (None, 16)                3136      
_________________________________________________________________
dense_2 (Dense)              (None, 10)                170       
Total params: 11,114
Trainable params: 11,114
Non-trainable params: 0
_________________________________________________________________


In [0]:
# 訓練の開始
epochs = 10
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))

# Evaluation.
scores = model.evaluate(x_test, y_test, verbose=True)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

Instructions for updating:
Use tf.cast instead.
Train on 10000 samples, validate on 10000 samples
Epoch 1/10
10000/10000 [==============================] - 13s 1ms/step - loss: 1.6350 - acc: 0.4900 - val_loss: 1.5843 - val_acc: 0.4688
Epoch 2/10
10000/10000 [==============================] - 11s 1ms/step - loss: 1.0421 - acc: 0.6866 - val_loss: 1.4250 - val_acc: 0.5405
Epoch 3/10
10000/10000 [==============================] - 11s 1ms/step - loss: 0.8423 - acc: 0.7438 - val_loss: 1.2842 - val_acc: 0.6049
Epoch 4/10
10000/10000 [==============================] - 12s 1ms/step - loss: 0.7212 - acc: 0.7808 - val_loss: 1.1883 - val_acc: 0.6280
Epoch 5/10
10000/10000 [==============================] - 11s 1ms/step - loss: 0.6391 - acc: 0.8020 - val_loss: 1.1172 - val_acc: 0.6548
Epoch 6/10
10000/10000 [==============================] - 11s 1ms/step - loss: 0.5761 - acc: 0.8213 - val_loss: 1.0130 - val_acc: 0.6896
Epoch 7/10
10000/10000 [==============================] - 12s 1ms/step - loss: 0